In [1]:
import _pickle as cPickle
import data_transformation as trans
import pandas as pd
import numpy as np

In [2]:
with open(r"trained_model_metrics.pickle", "rb") as input_file:
        trained_model_metrics = cPickle.load(input_file)
        
trained_model = trained_model_metrics["model"]
subMSE = trained_model_metrics["MSE"]
subRMSE = trained_model_metrics["RMSE"]
model_data = trained_model_metrics["data"]

## Prediction of Compressive Strentght of Given Mix Design

### Set the mix design

In [3]:
#Possible cement types
model_data["CemType"].unique()

array(['CEM I 52,5 R', 'CEM I 42,5 R', 'CEM II/A-M 42,5 R',
       'CEM II/B-S 32,5 R', 'CEM III/B 32,5 N SR'], dtype=object)

In [4]:
#Possible age from casting
model_data["Age"].unique()

array([ 1,  2,  3,  7,  8, 14, 28, 29, 90])

In [24]:
mix_design = {'CemType' : "CEM I 42,5 R",
              'CemAmt' : 400, 
              'FlyAsh' : 0,
              'Limestone' : 0, 
              'Microsil' : 0,
              'MicrosilSuspen' : 0, 
              'CleanWater' : 100, 
              'SludgeWater' : 80, 
              'Plast' : 0, 
              'Superplast' : 3.0,
              'AirEntrainer' : 0.6, 
              'Retarder' : 0, 
              'Accelerator' : 0, 
              'ShrinkageReducer' : 0,
              'Stabilizer': 0, 
              'Crystalizer': 0, 
              'SlumpRetention' : 0, 
              'Agg_0_1' : 0, 
              'Agg_0_2': 0,
              'Agg_0_4' : 780, 
              'Agg_4_6' : 0, 
              'Agg_4_8' : 0, 
              'Agg_8_16' : 1020, 
              'Agg_16_22' : 0, 
              'PPFibres' : 0,
              'GlassFibres' : 0, 
              "Age": 2
             }

### Processing of the Data

In [25]:
df = pd.DataFrame(mix_design,index=[0])

In [26]:
df = trans.DataTransformatorPipeline.sum_water_agg_admix_add_fibres_microsil(df)
df = trans.DataTransformatorPipeline.drop_insignificant_variables(df)
df = trans.DataTransformatorPipeline.make_log_columns(df,["CemAmt","TotalWater"])
df = trans.DataTransformatorPipeline.make_interactions(df,"Age",["CemType","Age"])

Variables were aggregated and selected dropped.
Insignificant variables ('ShrinkageReducer','Stabilizer','SlumpRetention','Crystalizer') were dropped.
['CemAmt', 'TotalWater'] variables were logarithmized and original variables dropped.
Interactions (product) of Age and all variables apart from ['CemType', 'Age'] were made and original variables dropped.


In [27]:
df.columns = [x for x in list(model_data.columns) if x not in ["ID","fc_log"]]

### Prediction of the Compressive Strength

In [28]:
prediction = np.exp(trained_model.predict(df))[0].round(2)
age = int(mix_design["Age"])
predicton_RMSE = subRMSE[age]

In [29]:
print(f"Predicted compressive strength for given mix design: {prediction} MPa")
print(f"Root mean squared error on the given day on model testing set was: {predicton_RMSE} MPa")

Predicted compressive strength for given mix design: 31.23 MPa
Root mean squared error on the given day on model testing set was: 5.27 MPa
